In [1]:
import os, re, glob, json
from os.path import join as opj
import numpy as np
import pandas as pd

import networkx as nx

import ohol_transitions as transition
import ohol_object as obj

In [2]:
a = !pwd
baseDir = '/'.join(a[0].split('/')[0:-2])
baseDir

'/Users/dengwenning/Documents/GitHub'

In [3]:
gsearch = lambda *args: glob.glob(opj(*args))
data_dir = '../../OneLifeData7/'
trans_dir  = opj(data_dir, 'transitions')
trans_files = gsearch(trans_dir, '*.txt')

print(*trans_files[:5], sep='\n')

../../OneLifeData7/transitions/-1_2574.txt
../../OneLifeData7/transitions/0_702.txt
../../OneLifeData7/transitions/314_235.txt
../../OneLifeData7/transitions/2165_2165.txt
../../OneLifeData7/transitions/0_1692.txt


In [4]:
tech_df = pd.DataFrame()
for i in trans_files:
    a = transition.read_transition(opj(trans_dir, i))
    tech_df = pd.concat([tech_df, pd.DataFrame.from_records(a, index=[0])], sort=True)

In [5]:
tech_df.head()

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,0.0,10,1.0,False,False,False,0.0,0,Player,2578,Cool Glass,0.0,0.0,-1,Empty,2574,Molten Glass,0.0,0.0,0.0
0,0.0,0,1.0,False,False,False,0.0,425,Wolf Skin,695,Wolf Crown,NaN,NaN,0,Player,702,Wolf Crown with Wolf Skin,0.0,0.0,0.0
0,NaN,0,NaN,False,False,False,NaN,0,Player,317,Crucible with Iron,NaN,NaN,314,Wrought Iron,235,Clay Bowl - empty,NaN,NaN,NaN
0,0.0,0,1.0,True,False,False,0.0,235,Clay Bowl - empty,3699,Bowl with Raw Rubber Balls,0.0,0.0,2165,Bowl with Raw Rubber Ball,2165,Bowl with Raw Rubber Ball,0.0,1.0,0.0
0,0.0,0,1.0,False,False,False,0.0,1719,Hungry Schnauser Puppy - held,1706,Schnauser with Puppies -2,0.0,0.0,0,Player,1692,Schnauser with Puppies -3,0.0,0.0,0.0


In [6]:
tech_df.query('origActor == 314')

,actorMinUseFraction,autoDecaySeconds,desiredMoveDist,isTool,lastUseActor,lastUseTarget,move,newActor,newActorName,newTarget,newTargetName,noUseActor,noUseTarget,origActor,origActorName,origTarget,origTargetName,reverseUseActor,reverseUseTarget,targetMinUseFraction
0,NaN,0,NaN,False,False,False,NaN,0,Player,317,Crucible with Iron,NaN,NaN,314,Wrought Iron,235,Clay Bowl - empty,NaN,NaN,NaN
0,NaN,0,NaN,False,False,False,NaN,0,Player,316,Crucible with Iron and Charcoal,NaN,NaN,314,Wrought Iron,318,Crucible with Charcoal,NaN,NaN,NaN


In [292]:
col = ['origActor','origTarget','newActor','newTarget']
tech_tree_df = tech_df[col]
tech_tree_df = tech_tree_df.query('(origActor >= 0) and (origTarget >= 0)').reset_index(drop = True)
tech_tree_df.head()

,origActor,origTarget,newActor,newTarget
0,0,702,425,695
1,314,235,0,317
2,2165,2165,235,3699
3,0,1692,1719,1706
4,0,3085,2228,3086


In [8]:
def read_obj_file(obj_id):
    obj_str = str(obj_id)
    ## Read object file\n",
    obj_file = baseDir + '/OneLifeData7/objects/%s.txt' % obj_str
    with open(obj_file, 'r') as file:
        raw_data = file.read().splitlines()

    return raw_data

In [27]:
#determine if this is a natural object
nat_obj = []
for i in range(1,len(tech_tree_df)):
    try:
        b = read_obj_file(i)[7].split('#')[0]
        if float(b.split('=')[1]) != 0:
            nat_obj.append(i)
    except:
        FileNotFoundError

In [296]:
max_item_id = max(np.max(tech_tree_df).to_list())
#replace negative values
tech_tree_df = tech_tree_df.replace(-1, max_item_id+1)
#make a list of all objects
items = np.arange(max_item_id+2)
print(max_item_id)

4347


In [305]:
#BruteForce Recursion. Very slow

# changed = True
# while changed:
#     changed = False
#     #loop over all items  
#     for item in items:
        
#         #loop over the dataframe
#         for i in range(0,len(tech_tree_df)):
#             item1 = tech_tree_df.iloc[i]['origActor']
#             item2 = tech_tree_df.iloc[i]['origTarget']
#             product1 = tech_tree_df.iloc[i]['newActor']
#             product2 = tech_tree_df.iloc[i]['newTarget']
            
#             if (item == product1) or (item == product2):
#                 #get out of while loop is the depth of the item does not change
#                 changed = (depth[item1]+depth[item2]+1) < depth[item]
#                 #update the depth of this item
#                 depth[item] = min(depth[item], depth[item1]+depth[item2]+1)

In [294]:
#creating a nested dictionary that stores what item plus what other item lead to what products
d = dict()
for item in items:
    d[item] = {}
    recipy = tech_tree_df.query('(origActor == @item) or (origTarget == @item)')
    if len(recipy):
        for j in range(0,len(recipy)):
            ingredients = recipy.iloc[j].tolist()[0:2]
            ingredient = np.delete(ingredients, np.where(ingredients == item)[0][0])
            products = recipy.iloc[j].tolist()[2:4]
            d[item][ingredient[0]] = products

In [324]:
#initialize depth
depth = {}
for item in items:
    if item in nat_obj:
        depth[item] = 0
    else:
        depth[item] = 1000
depth[0] = 0 #empty hand
depth[len(depth)-1] = 0 #empty ground

In [325]:
#create a root to start from
root = np.transpose(np.append(0,nat_obj))

while len(root):
    #take an item from root and remove it
    item = root[0]
    root = np.delete(root,0)
    print(root)
    #put all products associated with this item into root
    for key in d[item]: #looping through other ingredients
        for product in d[item][key]:
            if depth[product] > (depth[item] + depth[key] + 1):
                depth[product] = depth[item] + depth[key] + 1
                root = np.append(root, product)
                

[  30   32   33   34   36   49   50   63   65   67   68   69   70   71
   73   74   92   99  100  107  121  125  131  133  136  152  153  160
  181  190  192  197  210  211  242  285  314  326  334  418  441  455
  458  461  462  464  467  474  475  487  502  527  530  531  560  568
  583  674  680  684  703  707  713  714  729  749  750  760  761  764
  765  767  768  769  778  780  789  791  804  805  808  837  850  851
  857  858  859  900  930  946  961 1000 1013 1020 1093 1107 1114 1126
 1129 1132 1140 1148 1157 1184 1186 1203 1228 1294 1309 1323 1358 1368
 1378 1422 1435 1596 1617 1624 1890 1891 1956 2099 2103 2104 2105 2106
 2107 2108 2109 2113 2114 2115 2116 2117 2118 2130 2143 2144 2166 2168
 2170 2171 2178 2191 2192 2212 2219 2257 2266 2267 2291 2292 2355 2363
 2364 2368 2369 2371 2372 2373 2374 2375 2376 2379 2380 2381 2391 2396
 2400 2407 2413 2428 2429 2430 2431 2432 2433 2434 2443 2444 2445 2446
 2447 2448 2465 2467]
[  32   33   34   36   49   50   63   65   67   68   69

 1383 2169 2382 2392 2436 2437   58 3290  408 2419   62]
[  48  503  233  661  486  949  138 1254  150 2719  342 1829  340 1830
 3288  557 1221  708   75 2214 2691  579 1620 1832 3146  191 2435 3109
  151 1330 1438 1442  539  420 1831 2591  906   72  905 2746  465  508
 2886  556  546 3144 3145  801  681 1854 2401 2710 2412 1379 1384 1383
 2169 2382 2392 2436 2437   58 3290  408 2419   62]
[ 503  233  661  486  949  138 1254  150 2719  342 1829  340 1830 3288
  557 1221  708   75 2214 2691  579 1620 1832 3146  191 2435 3109  151
 1330 1438 1442  539  420 1831 2591  906   72  905 2746  465  508 2886
  556  546 3144 3145  801  681 1854 2401 2710 2412 1379 1384 1383 2169
 2382 2392 2436 2437   58 3290  408 2419   62]
[ 233  661  486  949  138 1254  150 2719  342 1829  340 1830 3288  557
 1221  708   75 2214 2691  579 1620 1832 3146  191 2435 3109  151 1330
 1438 1442  539  420 1831 2591  906   72  905 2746  465  508 2886  556
  546 3144 3145  801  681 1854 2401 2710 2412 1379 1384 1383 21

In [330]:
depth.values()

dict_values([0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0, 1, 0, 0, 0, 1000, 0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1, 1000, 1000, 1, 0, 0, 1000, 1000, 1, 1000, 1000, 1000, 1, 3, 7, 1000, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1000, 4, 6, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0, 1000, 1000, 1000, 1, 1000, 1000, 0, 0, 1000, 1000, 1000, 1000, 1000, 10, 0, 2, 3, 4, 15, 13, 14, 11, 14, 15, 16, 1000, 15, 1000, 0, 1000, 1000, 1000, 0, 1, 1000, 1000, 1000, 1000, 0, 1, 0, 1000, 1000, 0, 2, 1, 2, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 3, 1000, 1, 1, 0, 0, 1000, 1000, 1000, 1000, 1000, 1000, 0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1, 0, 1, 0, 1000, 1000, 1000, 1000, 0, 100